In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input, GRU
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)


tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

# read word2vec
# 
word_vec_dict = {}
with open('../crawl-300d-2M.vec') as f:
    first_line_flag = True
    for line in f:
        if first_line_flag:
            first_line_flag= False
            continue
        v_list = line.rstrip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')

EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

Using TensorFlow backend.


(159571, 6)
(159571, 150) (153164, 150)
thuranx 29059
llibya 139832
shenanigans 15810
hgilbert 137237
lamoron 179605
slaughtered 10252
2000000
Preparing embedding matrix
Null word embeddings: 3760


In [2]:
from sklearn.metrics import log_loss,accuracy_score
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(GRU(80, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')

def model done


In [3]:
from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 8
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(curr_x, curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(hold_out_x,hold_out_y), 
                  callbacks=callbacks_list)
        
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [4]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

train_pred,test_pred = kf_train(fold_cnt=4,rnd=4)
print(train_pred.shape,test_pred.shape)    


# 40000, 150
# final 0.040348153796 0.984148122153, pub 9849

Train on 119678 samples, validate on 39893 samples
Epoch 1/8
119678/119678 [==============================] - 487s 4ms/step - loss: 0.0582 - acc: 0.9795 - val_loss: 0.0443 - val_acc: 0.9827
Epoch 2/8
119678/119678 [==============================] - 487s 4ms/step - loss: 0.0434 - acc: 0.9834 - val_loss: 0.0416 - val_acc: 0.9838
Epoch 3/8
119678/119678 [==============================] - 489s 4ms/step - loss: 0.0406 - acc: 0.9843 - val_loss: 0.0406 - val_acc: 0.9838
Epoch 4/8
119678/119678 [==============================] - 491s 4ms/step - loss: 0.0383 - acc: 0.9850 - val_loss: 0.0411 - val_acc: 0.9837
Epoch 5/8
119678/119678 [==============================] - 493s 4ms/step - loss: 0.0370 - acc: 0.9856 - val_loss: 0.0410 - val_acc: 0.9837
Epoch 6/8
119678/119678 [==============================] - 494s 4ms/step - loss: 0.0351 - acc: 0.9862 - val_loss: 0.0409 - val_acc: 0.9839
Epoch 7/8
119678/119678 [==============================] - 497s 4ms/step - loss: 0.0337 - acc: 0.9867 - val_loss: 0

Epoch 5/8
119679/119679 [==============================] - 472s 4ms/step - loss: 0.0365 - acc: 0.9856 - val_loss: 0.0420 - val_acc: 0.9837
Epoch 6/8
119679/119679 [==============================] - 471s 4ms/step - loss: 0.0349 - acc: 0.9862 - val_loss: 0.0413 - val_acc: 0.9840
Epoch 7/8
119679/119679 [==============================] - 472s 4ms/step - loss: 0.0331 - acc: 0.9871 - val_loss: 0.0418 - val_acc: 0.9838
Epoch 8/8
119679/119679 [==============================] - 472s 4ms/step - loss: 0.0317 - acc: 0.9875 - val_loss: 0.0423 - val_acc: 0.9837
-------------------------------
0 0.0883629048591 0.96644753746
1 0.0223739609519 0.990248854742
2 0.0449250225548 0.982290015103
3 0.00800831313311 0.997280207557
4 0.0591202311593 0.975484267191
5 0.0192984901181 0.993137850863
final 0.040348153796 0.984148122153
all eval None
(159571, 6) (153164, 6)


In [5]:
sample_submission[list_classes] = test_pred
sample_submission.to_csv("../results/pool_gru1_fasttext_sample_4.gz", index=False, compression='gzip')
with open('../features/pool_gru_fasttext_4_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)
print(sample_submission.head())
print('===================================')

                 id     toxic  severe_toxic   obscene    threat    insult  \
0  00001cee341fdb12  0.994778      0.509820  0.970709  0.308434  0.931687   
1  0000247867823ef7  0.000976      0.000090  0.000451  0.000031  0.000396   
2  00013b17ad220c46  0.004889      0.000405  0.002275  0.000302  0.001919   
3  00017563c3f7919a  0.000238      0.000023  0.000092  0.000049  0.000141   
4  00017695ad8997eb  0.006597      0.000163  0.000951  0.000110  0.000669   

   identity_hate  
0       0.462867  
1       0.000082  
2       0.000348  
3       0.000018  
4       0.000103  
